Begiratu https://www-amdis.iaea.org/cgi-bin/ALADDIN/query.pl


In [14]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import warnings
from scipy.interpolate import griddata
from scipy import interpolate
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from GlobalModel_Class import GlobalModel
import Reactions as R

warnings.filterwarnings('ignore')

### Leer PEC y interpolar las lineas alfa, beta y gamma de Balmer (estan en funcion de ne y e)

In [15]:
f=open('PEC_H.dat')
boolean=True
line=f.readline()[:-1].split()
line=f.readline()[:-1].split()
excitations=[]
while boolean:
    if '/TYPE' not in line:
        boolean=False
    else :
        wavelenght=line[0]
        reac_type=line[8]
        Nde=int(line[1])
        Nte=int(line[2])
        nde=0
        nte=0
        den=[]
        te=[]
        data=np.zeros((Nde,Nte))
        while (nde<Nde):
            line=f.readline()[:-1].split()
            nde+=len(line)
            den=den+line
        den=np.array(den,dtype=float)
        while (nte<Nte):
            line=f.readline()[:-1].split()
            nte+=len(line)
            te=te+line
        te=np.array(te,dtype=float)
        nde=0
        for nde in range (Nde):
            nte=0
            while (nte<Nte):
                line=f.readline()[:-1].split()
                for j in range (len(line)):
                    data[nde,nte]=line[j]
                    nte+=1
        excitations.append([data,wavelenght,reac_type])
        line=f.readline()[:-1].split()

for i in range (len(excitations)):
    reaction=excitations[i]
    if reaction[2]=='EXCIT':
        if reaction[1]=='6561.9A':
            H_alfa=reaction
            H_alfa_int=interpolate.interp2d(den,te,H_alfa[0].T) 
        if reaction[1]=='4860.6A':
            H_beta=reaction
            H_beta_int=interpolate.interp2d(den,te,H_beta[0].T) 
        if reaction[1]=='4339.9A':
            H_gamma=reaction
            H_gamma_int=interpolate.interp2d(den,te,H_gamma[0].T) 
    if reaction[2]=='RECOM':
        if reaction[1]=='6561.9A':
            H_alfa_r=reaction
            H_alfa_r_int=interpolate.interp2d(den,te,H_alfa_r[0].T) 
        if reaction[1]=='4860.6A':
            H_beta_r=reaction
            H_beta_r_int=interpolate.interp2d(den,te,H_beta_r[0].T) 
        if reaction[1]=='4339.9A':
            H_gamma_r=reaction
            H_gamma_r_int=interpolate.interp2d(den,te,H_gamma_r[0].T)         
            

### Resultado del Modelo Global

In [25]:
Np,Nn=30,30
pmin,pmax=1e5,2e6
nmin,nmax=1e14,1e15
gm=GlobalModel(R.reactions)
nn,pp,nH2,nH,n1,n2,n3,ne,e=gm.TwoD_sol(pmin,pmax,nmin,nmax,Np,Nn)

# Simular el espectro

### PEC(ne,e) en funcion de la potencia y densidad de gas neutro:

In [26]:
%matplotlib widget
fig = plt.figure(figsize=(7,7))
ax = fig.gca(projection='3d')
H_alfa_map,H_beta_map,H_gamma_map=np.empty((Np,Nn)),np.empty((Np,Nn)),np.empty((Np,Nn))
ne=n1+n2+n3
for i in range (Np):
    for j in range (Nn):
        H_alfa_map[i,j]=H_alfa_int(ne[i,j],e[i,j])
        H_beta_map[i,j]=H_beta_int(ne[i,j],e[i,j])
        H_gamma_map[i,j]=H_gamma_int(ne[i,j],e[i,j])
        
ax.scatter(nn,pp,H_alfa_map,alpha=0.5,color='red',label='H_a')
ax.scatter(nn,pp,H_beta_map,alpha=0.5,color='blue',label='H_b')
ax.scatter(nn,pp,H_gamma_map,alpha=0.5,color='green',label='H_g')
plt.xlabel('$n_0$ (cm$^{-3}$)')
plt.ylabel('$P_\mu$ (W/m$^{3}$)')
plt.legend()

fig = plt.figure(figsize=(7,7))
ax = fig.gca(projection='3d')
ax.scatter(nn,pp,H_beta_map/H_alfa_map,alpha=0.5)
#ax.scatter(nn,pp,H_alfa_map/H_gamma_map,alpha=0.5)
plt.xlabel('$n_0$ (cm$^{-3}$)')
plt.ylabel('$P_\mu$ (W/m$^{3}$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, '$P_\\mu$ (W/m$^{3}$)')

# La emision de cada linea será ne * nH * PEC(ne,e).

In [27]:
%matplotlib widget
fig = plt.figure(figsize=(7,7))
ax = fig.gca(projection='3d')
H_alfa_map,H_beta_map,H_gamma_map=np.empty((xN,yN)),np.empty((xN,yN)),np.empty((xN,yN))
H_alfa_r_map,H_beta_r_map,H_gamma_r_map=np.empty((xN,yN)),np.empty((xN,yN)),np.empty((xN,yN))
ne=n1+n2+n3
for i in range (xN):
    for j in range (yN):
        H_alfa_map[i,j]=H_alfa_int(ne[i,j],e[i,j])
        H_beta_map[i,j]=H_beta_int(ne[i,j],e[i,j])
        H_gamma_map[i,j]=H_gamma_int(ne[i,j],e[i,j])
        H_alfa_r_map[i,j]=H_alfa_r_int(ne[i,j],e[i,j])
        H_beta_r_map[i,j]=H_beta_r_int(ne[i,j],e[i,j])
        H_gamma_r_map[i,j]=H_gamma_r_int(ne[i,j],e[i,j])
        
ax.scatter(nn,pp,(H_alfa_map*nH+H_alfa_r_map*n1)*ne,alpha=0.5,color='red',label='H_a intensity')
ax.scatter(nn,pp,(H_beta_map*nH+H_beta_r_map*n1)*ne,alpha=0.5,color='blue',label='H_b intensity')
ax.scatter(nn,pp,(H_beta_map*nH+H_beta_r_map*n1+H_alfa_map*nH+H_alfa_r_map*n1)*ne,alpha=0.5,color='purple',label='H_b+H_a intensity')

plt.xlabel('$n_0$ (cm$^{-3}$)')
plt.ylabel('$P_\mu$ (W/m$^{3}$)')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …